In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import glob
import os
from time import sleep
from tqdm import tqdm

In [ ]:
open_dir = './Batches/0/'

In [ ]:
def expandResults(wait,driver):    
    # Click expand button
    selector = '#results > div.bootstrap-table > div.fixed-table-container > div.fixed-table-pagination > div.pull-left.pagination-detail > span.page-list > span > button'
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector))) 
    driver.find_element_by_css_selector(selector).click()
    
    # Select
    selector = "#results > div.bootstrap-table > div.fixed-table-container > div.fixed-table-pagination > div.pull-left.pagination-detail > span.page-list > span > ul > li > a"
    options = driver.find_elements_by_css_selector(selector)
    options[-1].click()

In [ ]:
def downloadFITS(batchPath, driver, wait):
    # Clean text area
    driver.find_element_by_id('ra_dec_csv').clear()
    # Insert coords
    driver.find_element_by_id('ra_dec').send_keys(batchPath)
    # Search
    driver.find_element_by_id('bulk_search').click()
    
    # Check if there are results
    selector = '#bulk > fieldset > legend > span'
    elem = driver.find_element_by_css_selector(selector)
    numResults = int(elem.text.split(' ')[1])
    if numResults ==0:
        return
    
    # Wait for first results to be loaded
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#table > tbody > tr:nth-child(1) > td:nth-child(11) > a')))
    
    # Expand results fi necessary
    if numResults >10:
        expandResults(wait,driver)
    
    # Wait results view
    sleep(2)
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.fixed-table-loading')))
    
    # Download .fits
    elems = driver.find_elements_by_css_selector('#table > tbody > tr > td:nth-child(11) > a')
    print(str(numResults)+","+str(len(elems)))
    for e in elems:
        sleep(2)
        e.click()

In [ ]:
def retrieveURLS(open_dir):
    driver = webdriver.Chrome('./chromedriver')
    driver.get('https://dr14.sdss.org/optical/spectrum/search')
    
    # General settings
    # Bulk Search
    driver.find_element_by_id('select_bulk_search').click()
    wait = WebDriverWait(driver, 300)
    # Radius
    driver.find_element_by_id('radius').send_keys('0.02')
    # Decimal
    driver.find_element_by_css_selector('#option > div:nth-child(2) > div > div > ul > li > div > label:nth-child(2)').click() 
    
    files = glob.glob(open_dir+'*.txt')
    files = [os.path.abspath(f) for f in files]
    print(files)
    for batchPath in tqdm(files): 
        downloadFITS(batchPath, driver, wait)
        os.remove(batchPath)
    os.rmdir(open_dir)
    driver.close()

In [ ]:
retrieveURLS(open_dir)